In [1]:
import time
time.ctime()

'Mon Jun 25 13:49:01 2018'

# Prefix geolocation
This report checks for geolocation of all IPv4 prefixes assigned and allocated after 01 April 2018, date when the softlanding phase 1 started.

Rule
5.4.6.2 AFRINIC resources are for AFRINIC service region and any use outside the region should be solely in support of connectivity back to the AFRINIC region.

In [49]:
import numpy as np
import pandas as pd
import urllib, urllib2, json, re
from netaddr import *
from IPython.display import HTML

RIPE_STATS_URL = "https://stat.ripe.net/data/geoloc/data.json?resource="
IPRESOURCES_PATH = "ftp://ftp.afrinic.net/stats/afrinic/delegated-afrinic-extended-latest"

ccs = ['EG', 'BJ', 'CI', 'CV', 'GH', 'GM', 'GN', 'GW', 'AO', 'CF', 'CG', 'CM', 'GA', 
       'GQ', 'TD', 'BI', 'DJ', 'ER', 'ET', 'KM', 'BW', 'MA', 'SD', 'TN', 'LR', 'ML', 'MR', 'NE', 'NG', 'SL', 'SN', 'TG', 'ST', 'KE', 'MG', 'MU', 'MW', 'MZ', 'RE', 'RW', 'SC', 'SO', 'UG', 'LS', 'NA', 'SZ', 'ZA', 'DZ', 'EH', 'LY', 'BF', 'SH', 'CD', 'TZ', 'YT', 'ZM', 'ZW']

def getIPRange(prefix, prefixlength):
    
    #check for IPv6
    if ":" in prefix:
        return IPNetwork(prefix + "/" + str(prefixlength))
    else: 
        startip = IPAddress(prefix)
        endipint = int(startip) + int(prefixlength) -1
        endip = IPAddress(endipint)
        range = IPRange(startip, endip)
        return range.cidrs()


def getGeolocation(url):
    try:
        response = urllib2.urlopen(url)
        data = json.loads(response.read())
    except Exception, e:
        print e
    
    nb_loc = len(data['data']['locations'])
    
    return nb_loc, data['data']['locations']
    

In [50]:
df = pd.read_csv(IPRESOURCES_PATH, sep='|', 
                     skiprows=5, keep_default_na=0, 
                     names = ['rir','cc','type','resource','prefixlen','allocdate','status','opaqueid'])

#filter out unnecessary columns
df = df.iloc[:,[1,2,3,4,5,6]]
df['allocdate'] = pd.to_datetime(df['allocdate'])
df = df.loc[df['type'] == 'ipv4']
df = df.loc[df['allocdate'] > '2018-03-31']

df_found = pd.DataFrame(columns=['prefix', 'country', 'status', 'allocdate', 'locations', 'in_africa', 'out_africa'])

for index, prefix in df.iterrows():
    ip_range = getIPRange(prefix['resource'], prefix['prefixlen'])
    ip = str(ip_range[0])
    nb_loc, locations = getGeolocation(RIPE_STATS_URL + ip)
    
    if (nb_loc > 0):
        
        in_africa=0
        out_africa=0
        
        for loc in locations:
            if loc['country'] in ccs:
                in_africa = in_africa + 1
            else:
                out_africa = out_africa + 1
        
        df_found = df_found.append({'prefix': ip,'country': prefix['cc'] ,'status': prefix['status'], 
                                'allocdate': prefix['allocdate'], 'locations': nb_loc, 'in_africa': in_africa, 
                                    'out_africa' : out_africa}, ignore_index=True)

In [51]:
HTML(pd.DataFrame(df_found).to_html())

,prefix,country,status,allocdate,locations,in_africa,out_africa
0,196.49.52.0/24,TZ,assigned,2018-06-08,1,1,0
1,196.49.54.0/24,TZ,assigned,2018-06-08,1,1,0
2,196.49.56.0/24,CG,assigned,2018-06-18,1,1,0
3,196.60.50.0/24,TZ,assigned,2018-06-08,1,1,0
4,196.60.52.0/24,TZ,assigned,2018-06-08,1,1,0
